# Loading Trained Model

In [113]:
import torch
from torch import nn
from torch.utils.data import DataLoader

from Mymodel import MyResNet34
from Mymodel import MyResNet_CIFAR
import tqdm
import copy
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import datasets
from torchvision.transforms.v2 import (
    ToTensor,
    RandomHorizontalFlip,
    Compose,
    RandomCrop,
    RandomShortestSize,
    AutoAugment,
    Normalize,
    TenCrop,
    CenterCrop,
    Pad,
    Resize,
    Lambda
)
from torchvision.transforms.autoaugment import AutoAugmentPolicy

In [114]:
"""Dataset selection"""
# DATASET = "CIFAR10"
# DATASET = "CIFAR100"
DATASET = "ImageNet2012"

"""Model selection for CIFAR"""
NUM_LAYERS_LEVEL = 5

"""Dataset parameters"""
BATCH = 256
SHUFFLE = True
NUMOFWORKERS = 8
PIN_MEMORY = True
SPLIT_RATIO = 0

"""optimizer parameters"""
OPTIMIZER = "SGD"
# OPTIMIZER = "Adam"
# OPTIMIZER = "Adam_decay"

"""Learning rate scheduler parameters"""
# LOAD_BEFORE_TRAINING = False
LOAD_BEFORE_TRAINING = True
NUM_EPOCHS = 1000
scheduler_patience_mapping = {"CIFAR10": 100, "CIFAR100": 100, "ImageNet2012": 5}

"""Early stopping parameters"""
EARLYSTOPPINGPATIENCE = 25
file_path = ""
if DATASET == "ImageNet2012":
    file_path = f"{DATASET}/MyResNet34_{BATCH}_{OPTIMIZER}"
else:
    file_path = f"{DATASET}/MyResNet{NUM_LAYERS_LEVEL*6+2}_{BATCH}_{OPTIMIZER}"

if SPLIT_RATIO != 0:
    file_path += f"_{int(SPLIT_RATIO*100)}"

In [115]:
class LoadDataset:
    def __init__(self, root, seceted_dataset, split_ratio=0):
        self.Randp = 0.5
        self.dataset_name = seceted_dataset
        self.split_ratio = split_ratio

        if self.dataset_name[:5] == "CIFAR":
            pass
        elif self.dataset_name == "ImageNet2012":
            self.ImageNetRoot = root + "/" + self.dataset_name + "/"

            """
            각 지정된 스케일에 따라 10 crop해야하는데, 5개 scale들의 평균을 내야하니까 좀 번거로움.
            그치만, 학습 중엔 center crop으로 eval하니, 지금 당장 필요하지는 않음.
            """

            test_data_list = list()
            scales = [224, 256, 384, 480, 640]
            for scale in scales:
                # test_data_list.append(
                #     datasets.ImageFolder(
                #         root=self.ImageNetRoot + "val",
                #         transform=Compose(
                #             [
                #                 # RandomShortestSize(min_size=scale+1, antialias=True),
                #                 RandomShortestSize(min_size=scale+1, antialias=True),
                #                 # Resize(size=[scale, scale]),
                #                 TenCrop(size=scale),
                #                 ToTensor(),
                #                 Normalize(
                #                     mean=[0.485, 0.456, 0.406], std=[1, 1, 1], inplace=True
                #                 ),
                #             ]
                #         ),
                #     )
                # )
                test_data_list.append(
                    datasets.CIFAR100(
                        root=root,
                        train=True,
                        download=False,
                        transform=Compose(
                            [
                                # RandomShortestSize(min_size=scale+1, antialias=True),
                                # RandomShortestSize(min_size=scale+1, antialias=True),
                                # Resize(size=[scale, scale]),
                                ToTensor(),
                                TenCrop(size=20),
                                # Lambda(lambda crops: torch.stack(crops)),
                            ]
                        ),
                    )
                )
            self.test_data_list = test_data_list

        else:
            raise ValueError(f"Unsupported dataset: {self.dataset_name}")

        return

In [116]:
tmp = LoadDataset(root="../data", seceted_dataset=DATASET)
COUNT_OF_CLASSES = 1000
test_data = tmp.test_data_list

/home/lee/anaconda3/lib/python3.11/site-packages/torchvision/transforms/v2/_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [117]:
for i in range(5):
    print(test_data[i])

Dataset CIFAR100
    Number of datapoints: 50000
    Root location: ../data
    Split: Train
    StandardTransform
Transform: Compose(
                 ToTensor()
                 TenCrop(size=(20, 20), vertical_flip=False)
           )
Dataset CIFAR100
    Number of datapoints: 50000
    Root location: ../data
    Split: Train
    StandardTransform
Transform: Compose(
                 ToTensor()
                 TenCrop(size=(20, 20), vertical_flip=False)
           )
Dataset CIFAR100
    Number of datapoints: 50000
    Root location: ../data
    Split: Train
    StandardTransform
Transform: Compose(
                 ToTensor()
                 TenCrop(size=(20, 20), vertical_flip=False)
           )
Dataset CIFAR100
    Number of datapoints: 50000
    Root location: ../data
    Split: Train
    StandardTransform
Transform: Compose(
                 ToTensor()
                 TenCrop(size=(20, 20), vertical_flip=False)
           )
Dataset CIFAR100
    Number of datapoints: 50000
   

In [118]:
test_dataloader_list = list()

for i in range(5):
    test_dataloader_list.append(
        DataLoader(
            test_data[i],
            batch_size=BATCH,
            shuffle=SHUFFLE,
            # num_workers=NUMOFWORKERS,
            # pin_memory=PIN_MEMORY,
            # pin_memory_device="cuda",
            # persistent_workers=True,
        )
    )

In [119]:
for i in range(5):
    print(test_dataloader_list[i].dataset)

Dataset CIFAR100
    Number of datapoints: 50000
    Root location: ../data
    Split: Train
    StandardTransform
Transform: Compose(
                 ToTensor()
                 TenCrop(size=(20, 20), vertical_flip=False)
           )
Dataset CIFAR100
    Number of datapoints: 50000
    Root location: ../data
    Split: Train
    StandardTransform
Transform: Compose(
                 ToTensor()
                 TenCrop(size=(20, 20), vertical_flip=False)
           )
Dataset CIFAR100
    Number of datapoints: 50000
    Root location: ../data
    Split: Train
    StandardTransform
Transform: Compose(
                 ToTensor()
                 TenCrop(size=(20, 20), vertical_flip=False)
           )
Dataset CIFAR100
    Number of datapoints: 50000
    Root location: ../data
    Split: Train
    StandardTransform
Transform: Compose(
                 ToTensor()
                 TenCrop(size=(20, 20), vertical_flip=False)
           )
Dataset CIFAR100
    Number of datapoints: 50000
   

In [120]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if DATASET == "ImageNet2012":
    model = MyResNet34(
    num_classes=COUNT_OF_CLASSES, 
    Downsample_option="B"
    ).to(device)
    # model = models.resnet34(pretrained=True).to(device)
    # model = models.resnet34(pretrained=False).to(device)
    print(f"ResNet-34 for {DATASET} is loaded.")
else:
    model = MyResNet_CIFAR(
        num_classes=COUNT_OF_CLASSES, num_layer_factor=NUM_LAYERS_LEVEL
    ).to(device)


model.load_state_dict(torch.load(f"../models/{file_path}.pth"))

ResNet-34 for ImageNet2012 is loaded.


<All keys matched successfully>

In [121]:
for images, labels in tqdm.tqdm(test_dataloader_list[i], desc=f"test{i}"):
    print("-----------------------------------------")
    for img in images:
        print(img.shape)
    # print(len(images), len(labels))
    # print(images[0].shape, labels.shape)
    # for img in images:
    #     print(img.shape)
    #     img.to(device)
    # for label in labels:
    #     print(label)
    break

test4:   0%|          | 0/196 [00:00<?, ?it/s]

-----------------------------------------
torch.Size([256, 3, 20, 20])
torch.Size([256, 3, 20, 20])
torch.Size([256, 3, 20, 20])
torch.Size([256, 3, 20, 20])
torch.Size([256, 3, 20, 20])
torch.Size([256, 3, 20, 20])
torch.Size([256, 3, 20, 20])
torch.Size([256, 3, 20, 20])
torch.Size([256, 3, 20, 20])
torch.Size([256, 3, 20, 20])


In [125]:
criterion = nn.CrossEntropyLoss()
model.eval()
test_loss = 0.0
correct = 0
total = 0
avg_loss = 0
avg_acc = 0
for i in range(5):
    with torch.no_grad():
        for images, labels in tqdm.tqdm(test_dataloader_list[i], desc=f"test{i}"):
            for img in images:
                img, labels = img.to(device), labels.to(device)

                outputs = model(img)
                loss = criterion(outputs, labels)

                test_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

        test_loss /= len(test_dataloader_list[i])
        test_acc = correct / total
        print(test_loss, test_acc)
        avg_loss += test_loss
        avg_acc += test_acc

print(avg_loss / 5, avg_acc / 5)

test0:  47%|████▋     | 93/196 [00:15<00:13,  7.56it/s]

In [ ]:
print("test_loss:", test_loss)

In [ ]:
print(f"test_acc: {test_acc*100:.2f}%")


In [ ]:
print(f"test_error: {100 - test_acc*100:.2f}%")

In [ ]:
file_path